#### Imports

In [1]:
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime #used in Experian Knock-off checks specifically if there is <12mo credit history

### ① SEON Fraud API

### ② Experian API

#### Dummy Experian HTTP Endpoint

POST https://api.experian-mock.com/credit-report

In [ ]:
headers = {
    'X-API-KEY': 'xxxxxxxxxxxxx-xxxxxxxxxxxxxxx-xxxxxxxxxxxx'
}

payload = {
  "first_name": "Jonathan",
  "last_name": "Consumer",
  "ssn": "XXX-XX-2538",
  "date_of_birth": "1991-05-10",
  "address": {
    "street": "1475 Main St",
    "city": "Anytown",
    "state": "NY",
    "zip": "12345"
  }
}

response = requests.post('https://api.experian-mock.com/credit-report', headers=headers,
                         data=json.dumps(payload))
print(response.json())

#### Dummy Experian API Response (Created from PDF)

In [2]:
dummy_credit_report = {
    "personal_information": {
        "names": [
            {"name": "JON CONSUMER", "name_id": "3055"},
            {"name": "JONATHAN CONSUMER", "name_id": "25152"},
            {"name": "J CONSUMER", "name_id": "20726"}
        ],
        "addresses": [
            {
                "address": "1475 MAIN ST ANYTOWN USA 12345-1475",
                "address_id": "0122937323",
                "residence_type": "Single family",
                "geographical_code": "0-70010-17-2520"
            },
            {
                "address": "1036 MAIN ST APT143 ANYTOWN USA 12345-3043",
                "address_id": "0122868660",
                "residence_type": "Apartment complex",
                "geographical_code": "0-1020410-17-2520"
            }
        ],
        "ssn_variations": ["XXX-XX-2538", "XXX-XX-1680"],
        "year_of_birth": 1991,
        "spouse_or_co_applicant": "JANE",
        "notices": [
            "Your date of birth indicates that credit may have been established before age 18."
        ]
    },
    "personal_statements": [
        "FILE FROZEN DUE TO STATE LEGISLATION"
    ],
    "credit_accounts": {
        "negative_items": [
            {
                "account_name": "123 CREDIT CARDS",
                "account_number": "40034424804....",
                "address": "2000 MAIN ST ANYTOWN, USA 12345",
                "address_id": "0122868651",
                "type": "Credit card",
                "terms": "Revolving",
                "status": "Open",
                "recent_balance": {
                    "amount": 273,
                    "as_of": "06/03/2015"
                },
                "date_opened": "11/2013",
                "credit_limit": None,
                "high_balance": 14219,
                "monthly_payment": 10,
                "recent_payment_amount": 0,
                "past_due_amount": 20,
                "date_of_status": "06/2015",
                "first_reported": "12/2013",
                "responsibility": "Individual",
                "payment_history": {
                    "2015": {
                        "May": "30 days past due",
                        "Apr": "OK", "Mar": "OK", "Feb": "OK", "Jan": "OK"
                    }
                }
            }
        ],
        "accounts_in_good_standing": [
            {
                "account_name": "HOMETOWN AUTO",
                "recent_balance": 11616,
                "credit_limit": 19118,
                "high_balance": 19118,
                "monthly_payment": 350,
                "recent_payment_amount": 350,
                "date_opened": "03/2013",
                "status": "Current"
            },
            {
                "account_name": "AMERICAN APARTMENTS",
                "recent_balance": 4000,
                "credit_limit": 12000,
                "high_balance": 12000,
                "monthly_payment": 1000,
                "recent_payment_amount": 1000,
                "date_opened": "10/2014",
                "status": "Inactive/Never late"
            }
        ]
    }
}

### Knock-out Criteria Function 
**Input:** JSON response from Experian API (must be a 2xx response in format expected);
**Output:** Decision (Decline, Manual Review, Success) and Reason

In [5]:
def experian_report_eval(report):

    #KO Criteria #1: Is Applicants file frozen?
    for statement in report.get("personal_statements"):
        if "FILE FROZEN" in statement.upper(): #convert all statements to uppercase to avoid case sensitivity 
            return {
                    "decision": "Decline",
                    "reason":   "Applicants file is frozen"
            }


    #KO Criteria #2: Does Applicant have a credit past due balance?
    for account in report["credit_accounts"].get("negative_items"):
        past_due = account.get("past_due_amount", 0) #get the past due amount of the account
        
        #2a: If past_due > 10000 it is significant enough to reject applianct immediately
        if past_due > 10000:
            return {
                    "decision": "Decline",
                    "reason":   "Applicant has a credit account with past due > $10,000"
            }
            
        #2b: If past_due > 0 (but <10000) then should go through manual review (could be explainable reasons behind this)
        elif past_due > 0:
            return {
                    "decision": "Manual Review",
                    "reason":   "Applicant has a credit account with past due > $0, but < $10,000"
            }  

    #KO Criteria #3: Does Applicant have <12months of credit history?


    # If no past due accounts found
    return {
        "decision": "Continue Evaluation",
        "reason": "Passed credit check"
    }
    

In [9]:
experian_report_eval(dummy_credit_report)

{'decision': 'Decline', 'reason': 'Applicants file is frozen'}

### ③ Plaid API & Credit Determination